# IPython: beyond plain Python

When executing code in IPython, all valid Python syntax works as-is, but IPython provides a number of features designed to make the interactive experience more fluid and efficient.

## First things first: running code, getting help

In the notebook, to run a cell of code, hit `Shift-Enter`. This executes the cell and puts the cursor in the next cell below, or makes a new one if you are at the end.  Alternately, you can use:
    
- `Alt-Enter` to force the creation of a new cell unconditionally (useful when inserting new content in the middle of an existing notebook).
- `Control-Enter` executes the cell and keeps the cursor in the same cell, useful for quick experimentation of snippets that you don't need to keep permanently.

In [ ]:
print("Hi")

Getting help:

In [ ]:
?

Typing `object_name?` will print all sorts of details about any object, including docstrings, function definition lines (for call arguments) and constructor details for classes.

In [ ]:
import collections
collections.namedtuple?

In [ ]:
collections.Counter??

In [ ]:
*int*?

An IPython quick reference card:

In [ ]:
%quickref

## Tab completion

Tab completion, especially for attributes, is a convenient way to explore the structure of any object you’re dealing with. Simply type `object_name.<TAB>` to view the object’s attributes. Besides Python objects and keywords, tab completion also works on file and directory names.

In [ ]:
collections.

## The interactive workflow: input, output, history

In [ ]:
2+10

In [ ]:
_+10

You can suppress the storage and rendering of output if you append `;` to the last cell (this comes in handy when plotting with matplotlib, for example):

In [ ]:
10+20;

In [ ]:
_

The output is stored in `_N` and `Out[N]` variables:

In [ ]:
_10 == Out[10]

Previous inputs are available, too:

In [ ]:
In[11]

In [ ]:
_i

In [ ]:
%history -n 1-5

**Exercise**

Use `%history?` to have a look at `%history`'s magic documentation, and write the last 10 lines of history to a file named `log.py`.

## Accessing the underlying operating system

In [ ]:
!pwd

In [ ]:
files = !ls notebooks
print("files in notebooks directory:")
print(files)

In [ ]:
!echo $files

In [ ]:
!echo {files[0].upper()}

Note that all this is available even in multiline blocks:

In [ ]:
import os
for i,f in enumerate(files):
    if f.endswith('ipynb'):
        !echo {"%02d" % i} - "{os.path.splitext(f)[0]}"
    else:
        print('--')

## Beyond Python: magic functions

The IPyhton 'magic' functions are a set of commands, invoked by prepending one or two `%` signs to their name, that live in a namespace separate from your normal Python variables and provide a more command-like interface.  They take flags with `--` and arguments without quotes, parentheses or commas. The motivation behind this system is two-fold:
    
- To provide an orthogonal namespace for controlling IPython itself and exposing other system-oriented functionality.

- To expose a calling mode that requires minimal verbosity and typing while working interactively.  Thus the inspiration taken from the classic Unix shell style for commands.

In [ ]:
%magic

Line vs cell magics:

In [ ]:
%timeit list(range(1000))

In [ ]:
%%timeit
list(range(10))
list(range(100))

Line magics can be used even inside code blocks:

In [ ]:
for i in range(1, 5):
    size = i*100
    print('size:', size, end=' ')
    %timeit list(range(size))

Magics can do anything they want with their input, so it doesn't have to be valid Python:

In [ ]:
%%bash
echo "My shell is:" $SHELL
echo "My disk usage is:"
df -h

Another interesting cell magic: create any file you want locally from the notebook:

In [ ]:
%%writefile test.txt
This is a test file!
It can contain anything I want...

And more...

In [ ]:
!cat test.txt

Let's see what other magics are currently defined in the system:

In [ ]:
%lsmagic

In [ ]:
def to_optimize(N):
    total = [0,0]
    ta = 0
    tb = 0
    for i in range(N):
        for j in range(N):
            a = i**2
            b = j*2
            total[0] +=  a
            total[1] +=  b
    return total

In [ ]:
%timeit to_optimize(1_000)

In [ ]:
%prun to_optimize(1_000)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f to_optimize to_optimize(1_000)

## Running normal Python code: execution and errors

Not only can you input normal Python code, you can even paste straight from a Python or IPython shell session:

In [ ]:
>>> # Fibonacci series:
... # the sum of two elements defines the next
... a, b = 0, 1
>>> while b < 10:
...     print(b)
...     a, b = b, a+b

In [ ]:
In [1]: for i in range(10):
   ...:     print(i, end=' ')
   ...:     

And when your code produces errors, you can control how they are displayed with the `%xmode` magic:

In [ ]:
%%writefile mod.py

def f(x):
    return 1.0/(x-1)

def g(y):
    return f(y+1)

Now let's call the function `g` with an argument that would produce an error:

In [ ]:
import mod
mod.g(0)

In [ ]:
%xmode plain
mod.g(0)

In [ ]:
%xmode verbose
mod.g(0)

The default `%xmode` is "context", which shows additional context but not all local variables.  Let's restore that one for the rest of our session.

In [ ]:
%xmode context

## Running code in other languages with special `%%` magics

In [ ]:
%%perl
@months = ("July", "August", "September");
print $months[0];

In [ ]:
%%ruby
name = "world"
puts "Hello #{name.capitalize}!"

## Raw Input in the notebook

Since 1.0 the IPython notebook web application support `raw_input` which for example allow us to invoke the `%debug` magic in the notebook:

In [ ]:
mod.g(0)

In [ ]:
%debug

Don't foget to exit your debugging session. Raw input can of course be use to ask for user input:

In [ ]:
enjoy = input('Are you enjoying this tutorial? ')
print('enjoy is:', enjoy)

## Plotting in the notebook

This magic configures matplotlib to render its figures inline:

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = np.linspace(0, 2*np.pi, 300)
y = np.sin(x**2)
plt.plot(x, y)
plt.title("A little chirp")
fig = plt.gcf()  # let's keep the figure object around for later...

# Ventures into widgets

In [ ]:
from ipywidgets import interact, interact_manual

@interact(color=['C0','C1', 'C2'], f={'sine': np.sin, 'cos': np.cos})
def myplot(f, λ=2.0, color='C0'):
    x = np.linspace(0, 2*np.pi, 300)
    y = f(λ*x**2)
    plt.plot(x, y, c=color)
    plt.title(f"A little chirp (λ={λ})")
    